# calculate other_changes

In [65]:
import pandas as pd

# Load your data
df = pd.read_excel("output_cash_flow/all_update_combined.xlsx")

# Clean column names
df.columns = df.columns.str.strip()

# Convert all columns to numeric where possible, coercing errors to NaN
df = df.apply(pd.to_numeric, errors="coerce")


In [66]:
def compute_long_term_debt_principal_payments(row):
    original = row.get("long_term_debt_principal_payments")
    
    bonds = row.get("payments_on_bonds_payable", 0)
    notes = row.get("payments_on_notes_payable", 0)
    lease = row.get("payments_on_lease_liabilities", 0)

    # 替换 NaN 为 0
    bonds = 0 if pd.isna(bonds) else bonds
    notes = 0 if pd.isna(notes) else notes
    lease = 0 if pd.isna(lease) else lease

    total = bonds + notes + lease

    if total != 0:
        return total, True  # 用新的值覆盖
    else:
        return original, False  # 保留原值（哪怕是NaN）




In [67]:
df[["long_term_debt_principal_payments", "computed_flag"]] = df.apply(
    compute_long_term_debt_principal_payments, axis=1, result_type="expand"
)

In [68]:
# Compute other_changes_in_investment_activities, treating missing values as 0 during computation
def compute_other_changes(row):
    net_cash = row["net_cash_from_investment_activities"]
    capex = row["capital_expenses"]
    
    return (0 if pd.isna(net_cash) else net_cash) + (0 if pd.isna(capex) else capex)

df["other_changes_in_investment_activities"] = df.apply(compute_other_changes, axis=1)


$$
\begin{aligned}
\text{other\_changes\_in\_investment\_activities} =\ & 
\text{net\_cash\_from\_investment\_activities} \\
& +\ \text{capital\_expenses}
\end{aligned}
$$

In [69]:
# Compute other_changes_in_operating_activities, treating missing values as 0 during computation
def compute_operating_changes(row):
    net_cash = row["net_cash_from_operating_activities"]
    net_assets = row["total_change_in_net_assets"]
    non_cash = row["total_non_cash_exp"]
    working_cap = row["change_in_working_capital"]

    return (0 if pd.isna(net_cash) else net_cash) - (
        (0 if pd.isna(net_assets) else net_assets) +
        (0 if pd.isna(non_cash) else non_cash) +
        (0 if pd.isna(working_cap) else working_cap)
    )

df["other_changes_in_operating_activities"] = df.apply(compute_operating_changes, axis=1)


$$
\begin{aligned}
\text{other\_changes\_in\_operating\_activities} =\ & 
\text{net\_cash\_from\_operating\_activities} \\
& - \Big( 
\text{total\_change\_in\_net\_assets} 
+ \text{total\_non\_cash\_exp} 
+ \text{change\_in\_working\_capital}
\Big)
\end{aligned}
$$


In [70]:
# Compute change_in_long_term_debt, treating missing values as 0 during computation
def compute_change_in_long_term_debt(row):
    proceeds = row["long_term_debt_net_proceeds"]
    payments = row["long_term_debt_principal_payments"]

    return (0 if pd.isna(proceeds) else proceeds) - (0 if pd.isna(payments) else payments)

df["change_in_long_term_debt"] = df.apply(compute_change_in_long_term_debt, axis=1)



$$
\begin{aligned}
\text{change\_in\_long\_term\_debt} =\ & 
\text{long\_term\_debt\_net\_proceeds} \\
& -\ \text{long\_term\_debt\_principal\_payments}
\end{aligned}
$$

In [71]:
# Compute other_changes_in_financing_activities, treating missing values as 0 during computation
def compute_other_changes_in_financing(row):
    net_cash = 0 if pd.isna(row["net_cash_from_financing_activities"]) else row["net_cash_from_financing_activities"]
    debt_proceeds = 0 if pd.isna(row["long_term_debt_net_proceeds"]) else row["long_term_debt_net_proceeds"]
    debt_payments = 0 if pd.isna(row["long_term_debt_principal_payments"]) else row["long_term_debt_principal_payments"]
    noncapital_flows = 0 if pd.isna(row["cash_flows_from_noncapital_financing_activities"]) else row["cash_flows_from_noncapital_financing_activities"]
    capital_flows = 0 if pd.isna(row["cash_flows_from_capital_and_related_financing_activities"]) else row["cash_flows_from_capital_and_related_financing_activities"]

    debt_change = debt_proceeds - debt_payments
    financing_sum = debt_change + noncapital_flows + capital_flows

    return net_cash - financing_sum

df["other_changes_in_financing_activities"] = df.apply(compute_other_changes_in_financing, axis=1)





$$
\begin{aligned}
\text{other\_changes\_in\_financing\_activities} =\ & 
\text{net\_cash\_from\_financing\_activities} \\
& - \Big( 
\text{long\_term\_debt\_net\_proceeds} 
- \text{long\_term\_debt\_principal\_payments} \\
& \quad + \text{cash\_flows\_from\_noncapital\_financing\_activities} \\
& \quad + \text{cash\_flows\_from\_capital\_and\_related\_financing\_activities} 
\Big)
\end{aligned}
$$

In [72]:
# Save results
df.to_excel("output_cash_flow/all_update_combined_filled.xlsx", index=False)
